# MNIST Digit Classifier

In [1]:
# This cell assumes a project structure of: project-root/src/experiments/this_notebook.ipynb
# We append the parent directory to the system path, so now we can import modules from src
# We also create a variable named path which points to the project root.

import sys
from pathlib import Path

sys.path.append("../") # go to parent dir
path =  str(Path().resolve().parent.parent)

print(path)


/data2/Kaggle-Knowledge-Competitions


Class of configurations (consider using Hydra for heavier workloads in future):

In [2]:
from dataclasses import dataclass

@dataclass
class Configurations:
    data_dir: str = path + "/data/kaggle_mnist"
    batch_size: int = 50
    num_workers: int = 4

    lr: float = 0.01

    num_epochs: int = 2

    device: str = "cuda"
    log_dir: str = path + "/logs/torch-digit-classifier"
    log_every_n_steps: int = 100

cfg = Configurations()

In [3]:
import torch
import os
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard.writer import SummaryWriter

from trainer.digit_classifier_trainer import train_digit_classifier
from datasets.kaggle_mnist import KaggleMNIST
from models.digit_classifier import ResNet18

import socket
from datetime import datetime
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = os.path.join(
                cfg.log_dir, current_time + '_' + socket.gethostname())

device = torch.device(cfg.device)
model = ResNet18(in_channels=1, out_classes=10)
data = KaggleMNIST(data_dir=cfg.data_dir, train=True, transform=None)
optimizer = torch.optim.SGD(model.parameters(), lr=cfg.lr)
logger = SummaryWriter(log_dir=log_dir)

n_val = int(len(data) * 0.2)
n_train = len(data) - n_val
train_data, val_data = random_split(data, [n_train, n_val])

train_loader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
)

val_loader = DataLoader(
    val_data,
    shuffle=False,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
)

train_digit_classifier(
    model,
    train_loader,
    val_loader,
    num_epochs=cfg.num_epochs,
    device=device,
    optimizer=optimizer,
    logger=logger,
    log_every_n_steps=cfg.log_every_n_steps,
)

Epoch 1: val: 100%|██████████| 168/168 [00:01<00:00, 160.54it/s]


672

8400